In [45]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from collections import Counter
import time
import seaborn as sns
from matplotlib import pyplot as plt

import tensorflow as tf
#from keras.utils import to_categorical
import feather
import random
from datetime import *

import warnings
warnings.filterwarnings("ignore")

In [46]:
FILENAME = "Datasets/Round 3/preproccesed_outliers_131362_col_7_threshold_0.8_"
data = feather.read_dataframe("{}.feather".format(FILENAME))
data = data.dropna().reset_index(drop=True)

In [47]:
import censusdata

county = censusdata.download('acs5', 2018, censusdata.censusgeo([('county', '*')]),
                                   ['DP05_0001E',  'DP03_0019PE','DP03_0021PE','DP03_0022PE','DP03_0062E'],
                                   tabletype='profile')
# rename columns
county.columns = ['Population_County','Drive_County','Transit_County','Walk_County','MedianHouseholdIncome_County']
county = county.reset_index()
# extract county name and convert them to lowercase
county['County'] = county['index'].apply(lambda x : x.name.split(' County')[0].split(',')[0]).str.lower()
county['State'] = county['index'].apply(lambda x : x.name.split(', ')[1]).str.lower()
county

,index,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,County,State
0,"Washington County, Mississippi: Summary level:...",47086,86.4,0.0,1.3,30834,washington,mississippi
1,"Perry County, Mississippi: Summary level: 050,...",12028,85.8,0.0,1.8,39007,perry,mississippi
2,"Choctaw County, Mississippi: Summary level: 05...",8321,85.6,0.3,1.1,37203,choctaw,mississippi
3,"Itawamba County, Mississippi: Summary level: 0...",23480,82.4,0.2,0.7,40510,itawamba,mississippi
4,"Carroll County, Mississippi: Summary level: 05...",10129,90.0,0.0,1.4,43060,carroll,mississippi
...,...,...,...,...,...,...,...,...
3215,"Clayton County, Iowa: Summary level: 050, stat...",17672,78.6,0.3,4.3,52828,clayton,iowa
3216,"Buena Vista County, Iowa: Summary level: 050, ...",20260,73.0,0.3,5.6,54556,buena vista,iowa
3217,"Guthrie County, Iowa: Summary level: 050, stat...",10674,79.1,0.2,3.6,57075,guthrie,iowa
3218,"Humboldt County, Iowa: Summary level: 050, sta...",9566,84.8,0.0,1.8,52219,humboldt,iowa


In [48]:
# Convert all county values in data to lowercase

data['State'] = data['State'].str.lower() 
data['County'] = data['County'].str.lower()

counties = []
for i in data['County'].values:
    counties.append(i.replace(' county', ''))
data['County'] = counties

In [49]:
states = pd.read_csv('Datasets/states.csv')
states.index = states['Abbreviation'].map(lambda value: value.lower())
states = states.drop('Abbreviation',axis=1)
states

,State
Abbreviation,
al,Alabama
ak,Alaska
az,Arizona
ar,Arkansas
ca,California
co,Colorado
ct,Connecticut
de,Delaware
dc,District of Columbia


In [50]:
data['State'] = [states.loc[i, 'State'] for i in data['State'].values]
data['State'] = data['State'].map(lambda value: value.lower())

In [51]:
# Create join column <state>-<county>
def concat_cols(df, cols):
    df['combined'] = df[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
    return df

data = concat_cols(data, ['State', 'County'])
data.head()

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,combined
0,3,A-3,2.0,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-clermont
1,4,A-4,3.0,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-montgomery
2,5,A-5,2.0,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-montgomery
3,6,A-6,3.0,2016-02-08 07:44:26,2016-02-08 08:14:26,40.100590,-82.925194,0.01,Accident on I-270 Outerbelt Northbound near Ex...,Westerville Rd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-franklin
4,7,A-7,2.0,2016-02-08 07:59:35,2016-02-08 08:29:35,39.758274,-84.230507,0.00,Accident on Oakridge Dr at Woodward Ave. Expec...,N Woodward Ave,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-montgomery


In [52]:
county = concat_cols(county, ['State', 'County'])
county

,index,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,County,State,combined
0,"Washington County, Mississippi: Summary level:...",47086,86.4,0.0,1.3,30834,washington,mississippi,mississippi-washington
1,"Perry County, Mississippi: Summary level: 050,...",12028,85.8,0.0,1.8,39007,perry,mississippi,mississippi-perry
2,"Choctaw County, Mississippi: Summary level: 05...",8321,85.6,0.3,1.1,37203,choctaw,mississippi,mississippi-choctaw
3,"Itawamba County, Mississippi: Summary level: 0...",23480,82.4,0.2,0.7,40510,itawamba,mississippi,mississippi-itawamba
4,"Carroll County, Mississippi: Summary level: 05...",10129,90.0,0.0,1.4,43060,carroll,mississippi,mississippi-carroll
...,...,...,...,...,...,...,...,...,...
3215,"Clayton County, Iowa: Summary level: 050, stat...",17672,78.6,0.3,4.3,52828,clayton,iowa,iowa-clayton
3216,"Buena Vista County, Iowa: Summary level: 050, ...",20260,73.0,0.3,5.6,54556,buena vista,iowa,iowa-buena vista
3217,"Guthrie County, Iowa: Summary level: 050, stat...",10674,79.1,0.2,3.6,57075,guthrie,iowa,iowa-guthrie
3218,"Humboldt County, Iowa: Summary level: 050, sta...",9566,84.8,0.0,1.8,52219,humboldt,iowa,iowa-humboldt


### This is the code that destroys my RAM

In [53]:
# left join df with census data
data = data.merge(county, left_on = 'combined', right_on='combined',how = "left")
data

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,State_WY,combined,index,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,County_y,State_y
0,3,A-3,2.0,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.010,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,...,0.0,ohio-clermont,"Clermont County, Ohio: Summary level: 050, sta...",203216.0,85.4,0.4,1.1,64822.0,clermont,ohio
1,4,A-4,3.0,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.010,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,...,0.0,ohio-montgomery,"Montgomery County, Ohio: Summary level: 050, s...",532034.0,82.6,2.6,2.5,49314.0,montgomery,ohio
2,5,A-5,2.0,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.010,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,...,0.0,ohio-montgomery,"Montgomery County, Ohio: Summary level: 050, s...",532034.0,82.6,2.6,2.5,49314.0,montgomery,ohio
3,6,A-6,3.0,2016-02-08 07:44:26,2016-02-08 08:14:26,40.100590,-82.925194,0.010,Accident on I-270 Outerbelt Northbound near Ex...,Westerville Rd,...,0.0,ohio-franklin,"Franklin County, Ohio: Summary level: 050, sta...",1275333.0,81.3,2.3,2.5,58762.0,franklin,ohio
4,7,A-7,2.0,2016-02-08 07:59:35,2016-02-08 08:29:35,39.758274,-84.230507,0.000,Accident on Oakridge Dr at Woodward Ave. Expec...,N Woodward Ave,...,0.0,ohio-montgomery,"Montgomery County, Ohio: Summary level: 050, s...",532034.0,82.6,2.6,2.5,49314.0,montgomery,ohio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885011,3291539,A-3513775,2.0,2019-08-23 18:30:23,2019-08-23 18:58:54,34.239100,-118.416200,0.000,At Osborne St/Exit 154 - Accident.,Golden State Fwy N,...,0.0,california-los angeles,"Los Angeles County, California: Summary level:...",10098052.0,73.9,6.0,2.7,64251.0,los angeles,california
2885012,3291540,A-3513776,2.0,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379400,0.543,At Market St - Accident.,Pomona Fwy E,...,0.0,california-riverside,"Riverside County, California: Summary level: 0...",2383286.0,77.8,1.3,1.5,63948.0,riverside,california
2885013,3291541,A-3513777,2.0,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148100,0.338,At Camino Del Rio/Mission Center Rd - Accident.,I-8 W,...,0.0,california-san diego,"San Diego County, California: Summary level: 0...",3302833.0,76.3,2.9,2.9,74855.0,san diego,california
2885014,3291542,A-3513778,2.0,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847800,0.561,At Glassell St/Grand Ave - Accident. in the ri...,Garden Grove Fwy,...,0.0,california-orange,"Orange County, California: Summary level: 050,...",3164182.0,78.7,2.0,1.9,85398.0,orange,california


In [54]:
data = data.drop(["index","County_y","combined"],axis = 1)
data

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,State_WA,State_WI,State_WV,State_WY,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,State_y
0,3,A-3,2.0,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.010,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,...,0.0,0.0,0.0,0.0,203216.0,85.4,0.4,1.1,64822.0,ohio
1,4,A-4,3.0,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.010,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,...,0.0,0.0,0.0,0.0,532034.0,82.6,2.6,2.5,49314.0,ohio
2,5,A-5,2.0,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.010,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,...,0.0,0.0,0.0,0.0,532034.0,82.6,2.6,2.5,49314.0,ohio
3,6,A-6,3.0,2016-02-08 07:44:26,2016-02-08 08:14:26,40.100590,-82.925194,0.010,Accident on I-270 Outerbelt Northbound near Ex...,Westerville Rd,...,0.0,0.0,0.0,0.0,1275333.0,81.3,2.3,2.5,58762.0,ohio
4,7,A-7,2.0,2016-02-08 07:59:35,2016-02-08 08:29:35,39.758274,-84.230507,0.000,Accident on Oakridge Dr at Woodward Ave. Expec...,N Woodward Ave,...,0.0,0.0,0.0,0.0,532034.0,82.6,2.6,2.5,49314.0,ohio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885011,3291539,A-3513775,2.0,2019-08-23 18:30:23,2019-08-23 18:58:54,34.239100,-118.416200,0.000,At Osborne St/Exit 154 - Accident.,Golden State Fwy N,...,0.0,0.0,0.0,0.0,10098052.0,73.9,6.0,2.7,64251.0,california
2885012,3291540,A-3513776,2.0,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379400,0.543,At Market St - Accident.,Pomona Fwy E,...,0.0,0.0,0.0,0.0,2383286.0,77.8,1.3,1.5,63948.0,california
2885013,3291541,A-3513777,2.0,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148100,0.338,At Camino Del Rio/Mission Center Rd - Accident.,I-8 W,...,0.0,0.0,0.0,0.0,3302833.0,76.3,2.9,2.9,74855.0,california
2885014,3291542,A-3513778,2.0,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847800,0.561,At Glassell St/Grand Ave - Accident. in the ri...,Garden Grove Fwy,...,0.0,0.0,0.0,0.0,3164182.0,78.7,2.0,1.9,85398.0,california


In [55]:
data.to_feather("Datasets/combined_data.feather")